# 텍스트 마이닝을 활용한 영화 평점 감성 분석

## 1) 영화 평점 데이터 크롤링

In [25]:
import requests
from bs4 import BeautifulSoup
from urllib import parse

base_url = "https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=208077&target=after&page=2"
url = base_url.format(1)
res = requests.get(url)

soup = BeautifulSoup(res.text)
tds = soup.select('table.list_netizen > tbody > tr > td.title')

for td in tds:
    movie_title = td.select_one('a.movie').text.strip()
    score = td.select_one('div.list_netizen_score > em').text.strip()
    comment = td.select_one('br').next_sibling.strip()
    print(movie_title, score, comment, sep=' :: ')
    print('-------------------------------------------')

스파이더맨: 노 웨이 홈 :: 7 :: 쏘쏘~ 마블 취향이 아닌지라..
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 스파이더맨 전 시리즈를 안봤음에도 재미있었어요.
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 8 :: 액션은 부족했지만 스파이더맨팬에게는 선물 같은 영화였어요
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 보면 볼 수록 과거의 아련함이 묻어나온다.
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 2시간 영화를 봤는데, 본게 없다 .다시 봐야해
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 앤드류 갓필드...
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 6 :: 스토리 박살.  로맨스물. 평점엔 알바들만 득실. 소니 가 소니햇다.
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 7 :: 스파이더맨을 볼 수 밖에 없는 상영관이네요 ㅠㅠ
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 그냥 감동입니다 꼭보세요
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 10대의 나와 20대의 나..그리고 지금의 내가 만난듯한 기분
-------------------------------------------


In [44]:
import time
import random

comment_list=[] # 저장 할 리스트
for page in range(1, 1001):
    url = base_url.format(page)
    res = requests.get(url)
    
    soup = BeautifulSoup(res.text, 'lxml')
    tds = soup.select('table.list_netizen > tbody > tr > td.title')
    
    for td in tds:
        score = td.select_one('div.list_netizen_score > em').text.strip()
        comment = td.select_one('br').next_sibling.strip()
        comment_list.append((score, comment)) # 리스트에 추가
    delay = round(random.uniform(0.2, 1.2), 2) # 크롤링을 위해 페이지 전환을 빠르게 하게되면 네이버에서 막을 수 있기에 delay를 걸어준다.
    time.sleep(delay)
    

In [45]:
print(comment_list)

[('10', '정말 재미있게봤어요.'), ('10', '스파이더맨의 새로운 세계관이 눈물나는 반가움을 선물해주네요. 철부지 좌충우돌 십대 히어로가 고독한 성인 영웅으로 거듭나는 나름 감동적인 성장담.'), ('10', '역대급이다.. 진심 마블 영화중에 인피니티 다음으로 마음을 울렸다.. 내가 어릴때 토비 스파이더맨일때부터 스파이더맨 시리즈는 다 봤어서 그런지 여운도 많이 남고 무언가 감동도 많이 느끼고 정말 정말 재밌게 보았다.. 진심이다. 그리고 앞으로가 더욱 기대된다.'), ('10', ''), ('10', '사랑해요 다정한 스파이더맨 ..'), ('10', '문화적으로 또는 자본적으로 이루어진 우리시대의 "찐추억"'), ('8', ''), ('10', '폭로, 사투, 만남, 이별, 만남, 각성, 사투, 망각, 새출발'), ('10', '말로 표현 할수 없을 정도 임 N차 관람함...'), ('9', '토비 맥과이어를 다시 보게될 줄이야 소름 돋았다'), ('10', '정말 재미있게봤어요.'), ('10', '스파이더맨의 새로운 세계관이 눈물나는 반가움을 선물해주네요. 철부지 좌충우돌 십대 히어로가 고독한 성인 영웅으로 거듭나는 나름 감동적인 성장담.'), ('10', '역대급이다.. 진심 마블 영화중에 인피니티 다음으로 마음을 울렸다.. 내가 어릴때 토비 스파이더맨일때부터 스파이더맨 시리즈는 다 봤어서 그런지 여운도 많이 남고 무언가 감동도 많이 느끼고 정말 정말 재밌게 보았다.. 진심이다. 그리고 앞으로가 더욱 기대된다.'), ('10', ''), ('10', '사랑해요 다정한 스파이더맨 ..'), ('10', '문화적으로 또는 자본적으로 이루어진 우리시대의 "찐추억"'), ('8', ''), ('10', '폭로, 사투, 만남, 이별, 만남, 각성, 사투, 망각, 새출발'), ('10', '말로 표현 할수 없을 정도 임 N차 관람함...'), ('9', '토비 맥과이어를 다시 보게될 줄이야 소름 돋았다'), ('10', '정말 재미있게봤어요.'), 

In [46]:
import pandas as pd
df = pd.DataFrame(comment_list,columns=['rate','comment'])
df.to_csv('spider_naver_comment.csv', encoding='utf-8', index = False)

## 2) 텍스트 전처리를 위한 데이터 셋 살펴보기

In [37]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("https://raw.githubusercontent.com/nexussord/HUFS_MLP/main/spider_naver_comment.csv")

In [38]:
df.head()

,rate,comment
0,7,쏘쏘~ 마블 취향이 아닌지라..
1,10,스파이더맨 전 시리즈를 안봤음에도 재미있었어요.
2,8,액션은 부족했지만 스파이더맨팬에게는 선물 같은 영화였어요
3,10,보면 볼 수록 과거의 아련함이 묻어나온다.
4,10,"2시간 영화를 봤는데, 본게 없다 .다시 봐야해"


In [39]:
df.shape

(1000, 2)

In [40]:
df.isnull().sum() # 데이터 무결성 확인

rate       0
comment    0
dtype: int64

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   rate     1000 non-null   int64 
 1   comment  1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [42]:
df['comment'][0]

'쏘쏘~ 마블 취향이 아닌지라..'

In [43]:
len(df['comment'].values.sum())

29585

## 3) 한국어 텍스트 전처리

In [ ]:
!pip install konlpy==0.5.1 jpype1 